# WebLog Challenge

## Sessionize by IP

### Clean and create temporary folder.  Unzip data.

In [1]:
import shutil
import os
import gzip

scriptpath = os.getcwd()
tempdir = '.tmp'

if os.path.exists(tempdir):
    shutil.rmtree(tempdir)
os.makedirs(tempdir)

extractedlog = 'web.log'
sourcepath = os.path.join(scriptpath, '../data/2015_07_22_mktplace_shop_web_log_sample.log.gz')
destpath = os.path.join(scriptpath, tempdir, extractedlog)
with gzip.open(sourcepath) as src, open(destpath, 'w') as dest:
    content = src.read()
    dest.write(content)

## Build sessions

In [ ]:
from analytics.sessionizer import *
from pyspark import SparkContext

sessionizer = Sessionizer(sc)
sessions_rdd = sessionizer.calc_sessions_from_file(destpath, 15)

### Display first 10 sessions

In [ ]:
sessions_rdd.take(10)

## Average Session Time

In [ ]:
avg = sessionizer.average_session_time(sessions_rdd)

print 'Average session time(s):' + str(avg)

## Unique Visits per Sesions

In [ ]:
visits_per_session = sessionizer.unique_visits_per_session(sessions_rdd)

for session in visits_per_session[:10]:
    print 'Session ID:' + str(session['id']) + ', IP:' + session['ip'] + ', Number of Visits:' + str(session['visits']) 

## Active Users

In [ ]:
most_active_sessions = sessionizer.find_engaged_users(sessions_rdd)

for session in most_active_sessions[:10]:
    print 'User:' + session['ip'] + ', Duration(s):' + str(session['length']) 